In [120]:
%serialconnect to --port="/dev/tty.usbserial-02EDKZTP" --baud=115200

Connecting to --port=/dev/tty.usbserial-02EDKZTP --baud=115200 
MicroPython v1.14 on 2021-04-12; ESP32 module with ESP32
Type "help()" for more information.
>>>[reboot detected 0]repl is in normal command mode
[\r\x03\x03] b'\r\n>>> '
[\r\x01] b'\r\n>>> \r\nraw REPL; CTRL-B to exit\r\n>' Ready.


In [107]:
from ulab import numpy as np
import machine
from machine import Pin, PWM

In [5]:
import utime

led = Pin(22, Pin.OUT)

for i in range(10):
    led.on()
    utime.sleep(0.1)
    led.off()
    utime.sleep(0.1)

In [5]:
from lib.scheduling import LedFlasher

flasher = LedFlasher(0, 7, led)
flasher.start()

In [6]:
flasher.stop()

In [6]:
from lib.computation import solve_gen_eig_prob

M = np.eye(3)*np.array([-1, 1, 2])


A = np.array([[3.76405235, 0.        , 0.        ],
       [0.        , 2.40015721, 0.        ],
       [0.        , 0.        , 2.97873798]])

B = np.array([[4.2408932 , 0.        , 0.        ],
       [0.        , 3.86755799, 0.        ],
       [0.        , 0.        , 1.02272212]])


print(solve_gen_eig_prob(A, B))

(array([0.8875613, 0.6205873, 2.912558], dtype=float32), array([[0.4855918, 0.0, 0.0],
       [0.0, 0.508489, 0.0],
       [0.0, 0.0, 0.9888289]], dtype=float32))


In [108]:
import urandom

def synth_x(f, Ns, noise_power=0.5, fs=250):
    """
    generate a synthetic signal vector
    
    args:
    Ns [int]: number of samples (time samples)
    noise_power [float]: variance of WGN noise distribution
    """
    t = np.arange(0, Ns/fs, 1/fs)
    return np.sin(t*2*np.pi*f)+urandom.random()

def synth_X(f, Nc, Ns, noise_power=0.5, fs=200, f_std=0.02, noise_std=0.2):
    """
    Generate a matrix of several variations of the same target signal. This is used
    to simulate the measurement of a common signal over multiple EEG channels 
    that have different SNR characteristics.
    
    args:
    f [float]: target frequency of synthetic signal (Hz)
    Nc [int]: number of channels
    Ns [int]: number of samples (time samples)
    noise_power [float]: variance of WGN noise distribution
    fs [float]: sampling frequency (Hz)
    f_std [float]: standard dev. of freq. in generated signal across channels to simulate interference from other frequency components over different channels
    noise_std [float]: standard dev. of noise across channels
    """
    X = []
    for i in range(Nc): # simulate noisy sinusoids with varying SNR across Nc channels
        f_i = f*(1+urandom.random())
        sigma_i = noise_power*(1+urandom.random())
        x = synth_x(f_i, Ns, noise_power=sigma_i)
        
        x += synth_x(f_i*1.05, Ns)*0.2 # add extraneous neighbouring signals (task unrelated)
        x += synth_x(f_i*1.1, Ns)*0.1
        X.append(x)
        
    return np.array(X)

In [119]:
from lib.decoding import harmonic_reference
from lib.computation import solve_gen_eig_prob
from ulab import numpy as np

class CCA():
    
    def __init__(self, stim_freqs, fs, Nh=3):
        self.Nh = Nh
        self.stim_freqs = stim_freqs
        self.fs = fs
        
    def compute_corr(self, X_test):            
        result = {}
        for f in self.stim_freqs:
            Y = harmonic_reference(f, self.fs, np.max(X_test.shape()), Nh=self.Nh, standardise_out=True)
            rho = self.cca_eig(X_test, Y)[0] # canonical variable matrices. Xc = X^T.W_x
            result[f] = rho
        return result
    
    @staticmethod
    def cca_eig(X, Y, n_components=1):
        Cxx = np.dot(X, X.transpose()) # auto correlation matrix
        Cyy = np.dot(Y, Y.transpose()) 
        Cxy = np.dot(X, Y.transpose()) # cross correlation matrix
        Cyx = np.dot(Y, X.transpose()) # same as Cxy.T

        M1 = np.dot(np.linalg.inv(Cxx), Cxy) # intermediate result
        M2 = np.dot(np.linalg.inv(Cyy), Cyx)

        M = np.dot(M1, M2)
        lam, _ = solve_gen_eig_prob(M, np.eye(len(M))) #np.linalg.eig(M)
        
        return sorted(np.sqrt(lam), reverse=True)[:n_components] # return largest n sqrt eig vals

gc.collect()

f0 = 7
stim_freqs = [7, 10, 12]
fs = 200

X_test = synth_X(f0, 4, 100)

cca = CCA(stim_freqs, fs)

cca.compute_corr(X_test)


Traceback (most recent call last):
  File "<stdin>", line 45, in <module>
  File "<stdin>", line 16, in compute_corr
  File "<stdin>", line 31, in cca_eig
  File "computation.py", line 22, in solve_gen_eig_prob
ValueError: input matrix is asymmetric


In [ ]:
Y = harmonic_reference(f, 200, len(X_test), Nh=self.Nh, standardise_out=True)

In [114]:
Y = harmonic_reference(f0, fs, 200)
from lib.computation import standardise

print(standardise(Y))

array([[0.3085011, 0.6021429, 0.8667814, ..., -0.6021446, -0.3085003, 3.712228e-06],
       [1.380154, 1.279618, 1.117448, ..., 1.279618, 1.380155, 1.414213],
       [0.6021429, 1.08967, 1.369783, ..., -1.089672, -0.6021408, 7.674105e-06],
       [1.279619, 0.9014534, 0.3517004, ..., 0.9014505, 1.279619, 1.414213]], dtype=float32)


In [38]:
del X, Y

In [34]:
import gc
import micropython

gc.collect()

# X = synth_X(10, 4, 1000)
print(micropython.mem_info())

stack: 736 out of 15360
GC: total: 111168, used: 19168, free: 92000
 No. of 1-blocks: 57, 2-blocks: 25, max blk sz: 500, max free sz: 2140
None


In [ ]:
%ls

In [1]:
%ls --recursive lib

No serial connected
  %serialconnect to connect
  %esptool to flash the device
  %lsmagic to list commands

In [ ]:
%sendtofile lib/computation.mpy --source lib/computation.mpy

In [238]:
%sendtofile lib/decoding.py --source lib/decoding.py

Sent 55 lines (1976 bytes) to lib/decoding.py.


In [5]:
%rebootdevice

repl is in normal command mode
[\r\x03\x03] b'\r\nMicroPython v1.14 on 2021-04-12; ESP32 module with ESP32\r\nType "help()" for more information.\r\n>>> \r\n>>> \r\nMPY: soft reboot\r\nTraceback (most recent call last):\r\n  File "main.py", line 8, in <module>\r\nKeyboardInterrupt: \r\nMicroPython v1.14 on 2021-04-12; ESP32 module with ESP32\r\nType "help()" for more information.\r\n>>> \r\n>>> '
[\r\x01] b'\r\n>>> \r\nraw REPL; CTRL-B to exit\r\n>'

In [20]:
%lsmagic

%capture [--quiet] [--QUIET] outputfilename
    records output to a file

%comment
    print this into output

%disconnect [--raw]
    disconnects from web/serial connection

%esptool [--port PORT] {erase,esp32,esp8266} [binfile]
    commands for flashing your esp-device

%fetchfile [--binary] [--print] [--load] [--quiet] [--QUIET]
                  sourcefilename [destinationfilename]
    fetch and save a file from the device

%ls [--recurse] [dirname]
    list files on the device

%lsmagic
    list magic commands

%mpy-cross [--set-exe SET_EXE] [pyfile]
    cross-compile a .py file to a .mpy file

%readbytes [--binary]
    does serial.read_all()

%rebootdevice
    reboots device

%sendtofile [--append] [--mkdir] [--binary] [--execute] [--source [SOURCE]] [--quiet]
                   [--QUIET]
                   [destinationfilename]
    send cell contents or file/direcectory to the device

%serialconnect [--raw] [--port PORT] [--baud BAUD] [--verbose]
    connects to a device over US